In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, SplineTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [2]:
df = pd.read_csv("../files/BRISTOR_Zegoland_all_%.csv")

df['Date'] = pd.to_datetime(df['Date'])

df.set_index('Date', inplace=True)

In [14]:
X=df.iloc[:,8:13]
y=df.iloc[:,4:5]

data=pd.concat([y,X],axis=1)

data.head()

,Value,DemandShareA,newPatientShareA,newPatientShareB,totalPatientShareA,totalPatientShareB
Date,,,,,,
2021-11-01,56305.84,0.013829,0.05,0.001,0.01,0.000
2021-12-01,35581.00,0.013842,0.06,0.021,0.01,0.001
2022-01-01,41136.88,0.013942,0.05,0.026,0.01,0.002
2022-02-01,41689.80,0.014830,0.04,0.041,0.02,0.004
2022-03-01,54587.28,0.016513,0.05,0.028,0.02,0.005


In [20]:
# List of columns to create lags for
columns_to_lag = ['Value', 'DemandShareA']

num_lags=2
# Loop through each column and lag
for col in columns_to_lag:
    for lag in range(1, num_lags + 1):
        data[f'{col}_Lag{lag}'] = data[col].shift(lag)

In [22]:
data.head()

,Value,DemandShareA,newPatientShareA,newPatientShareB,totalPatientShareA,totalPatientShareB,Value_Lag1,Value_Lag2,DemandShareA_Lag1,DemandShareA_Lag2
Date,,,,,,,,,,
2021-11-01,56305.84,0.013829,0.05,0.001,0.01,0.000,NaN,NaN,NaN,NaN
2021-12-01,35581.00,0.013842,0.06,0.021,0.01,0.001,56305.84,NaN,0.013829,NaN
2022-01-01,41136.88,0.013942,0.05,0.026,0.01,0.002,35581.00,56305.84,0.013842,0.013829
2022-02-01,41689.80,0.014830,0.04,0.041,0.02,0.004,41136.88,35581.00,0.013942,0.013842
2022-03-01,54587.28,0.016513,0.05,0.028,0.02,0.005,41689.80,41136.88,0.014830,0.013942


In [28]:
data['Value_Lag1'] = data['Value_Lag1'].fillna(data['Value_Lag1'].mean())

data['Value_Lag2'] = data['Value_Lag2'].fillna(data['Value_Lag2'].mean())

data['DemandShareA_Lag1'] = data['DemandShareA_Lag1'].fillna(data['DemandShareA_Lag1'].mean())

data['DemandShareA_Lag2'] = data['DemandShareA_Lag2'].fillna(data['DemandShareA_Lag2'].mean())

In [29]:
data.head()

,Value,DemandShareA,newPatientShareA,newPatientShareB,totalPatientShareA,totalPatientShareB,Value_Lag1,Value_Lag2,DemandShareA_Lag1,DemandShareA_Lag2
Date,,,,,,,,,,
2021-11-01,56305.84,0.013829,0.05,0.001,0.01,0.000,68082.696552,68082.696552,0.021218,0.021035
2021-12-01,35581.00,0.013842,0.06,0.021,0.01,0.001,68082.696552,68082.696552,0.013829,0.021035
2022-01-01,41136.88,0.013942,0.05,0.026,0.01,0.002,56305.840000,56305.840000,0.013842,0.013829
2022-02-01,41689.80,0.014830,0.04,0.041,0.02,0.004,35581.000000,35581.000000,0.013942,0.013842
2022-03-01,54587.28,0.016513,0.05,0.028,0.02,0.005,41136.880000,41136.880000,0.014830,0.013942


In [31]:
def split_dataset(data):
    #split into standard weeks
    train, test = data[1:-5], data[-5:-1]
    

    return train, test

In [32]:
train_data, test_data = split_dataset(data)

In [37]:
y_train,X_train  = train_data.iloc[:,0:1], train_data.iloc[:,1:10]

y_test,X_test  = test_data.iloc[:,0:1], test_data.iloc[:,1:10]

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# Initialize XGBoost model
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3
)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate performance
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')